# Installing Pyspark

In [ ]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 22.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=27aef6c930ea8766f198b197cdf13c9f45e06ea338a5a116ae1e3be67c40b716
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


# creating spark Session

In [ ]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Practice').getOrCreate()
spark

# Creating dataFrame from hdfs Location

 reading data from hdfs remote server

df = spark.read.format('csv').load('hdfs://172.16.38.131.8020/bigdata/cse/app_prod/cse.app_prod.csv')

# spark Defult parallelism

spark will read data from hdfs source as 10 partitions ,user can defined default parallelism in spark

df = spark.read.format('csv').load('hdfs://172.16.38.131.8020/bigdata/cse/app_prod/cse.app_prod.csv',10)

#  Why Python slower then scala ?


python objects are converted into java (JVM) objects using  Py4j librery vice-versa .

scala objects are  By defalut java (JVM) objects.

Hence No convertion needed, so scala is faster than oython

# Spark1 Vs Spark2




**spark1:**
1.   permenent tables not created in spark 1 without hive integration but In
2.   we can only create spark context

**spark2:**

1.   Spark2 we can create permenent table without hive integration 
2.   On Single Node Spark cluster from Spark 2 Data stored in Local file system  & MetaData will stored in embedded berby database
3.   we can create both spark session and spark context








# Reading DataFrame from csv

In [ ]:
df = spark.read.format('csv').option('header',True).option('inferSchema',True).option('nullValue','null').load("/content/emp.csv")
df.show(5)
df.printSchema()

+-----+------+--------+----+----------+----+----+------+------------+
|EMPNO| ENAME|     JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|
+-----+------+--------+----+----------+----+----+------+------------+
| 7369| SMITH|   CLERK|7902|17-12-1980| 800| 300|    10|  01-01-2022|
| 7499| ALLEN|SALESMAN|7698|20-02-1981|1600| 300|    20|  01-01-2022|
| 7521|  WARD|SALESMAN|7698|22-02-1981|1250| 500|    30|  01-01-2022|
| 7566| JONES| MANAGER|7839|04-02-1981|2975|null|    40|  05-01-2022|
| 7654|MARTIN|SALESMAN|7698|21-09-1981|1250|1400|    50|  03-01-2022|
+-----+------+--------+----+----------+----+----+------+------------+
only showing top 5 rows

root
 |-- EMPNO: integer (nullable = true)
 |-- ENAME: string (nullable = true)
 |-- JOB: string (nullable = true)
 |-- MGR: integer (nullable = true)
 |-- HIREDATE: string (nullable = true)
 |-- SAL: integer (nullable = true)
 |-- COMM: integer (nullable = true)
 |-- DEPTNO: integer (nullable = true)
 |-- UPDATED_DATE: string (nullable = true

# convert from String (dd-mm-yyyy) date format to spark date format (yyyy-mm-dd)

In [ ]:
from pyspark.sql.functions import *


df1 = df.withColumn('HIREDATE',to_date('HIREDATE','dd-mm-yyyy')).withColumn('UPDATED_DATE',to_date('UPDATED_DATE','dd-mm-yyyy'))

df1.printSchema()

root
 |-- EMPNO: integer (nullable = true)
 |-- ENAME: string (nullable = true)
 |-- JOB: string (nullable = true)
 |-- MGR: integer (nullable = true)
 |-- HIREDATE: date (nullable = true)
 |-- SAL: integer (nullable = true)
 |-- COMM: integer (nullable = true)
 |-- DEPTNO: integer (nullable = true)
 |-- UPDATED_DATE: date (nullable = true)



# filter or Where operations

In [ ]:
df1.where('ENAME="SMITH"').show()

+-----+-----+-----+----+----------+---+----+------+------------+
|EMPNO|ENAME|  JOB| MGR|  HIREDATE|SAL|COMM|DEPTNO|UPDATED_DATE|
+-----+-----+-----+----+----------+---+----+------+------------+
| 7369|SMITH|CLERK|7902|1980-01-17|800| 300|    10|  2022-01-01|
| 7369|SMITH|CLERK|7902|1980-01-17|800|null|    40|  2022-01-04|
+-----+-----+-----+----+----------+---+----+------+------------+



In [ ]:
df1.filter('JOB = "MANAGER"').show()

+-----+-----+-------+----+----------+----+----+------+------------+
|EMPNO|ENAME|    JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|
+-----+-----+-------+----+----------+----+----+------+------------+
| 7566|JONES|MANAGER|7839|1981-01-04|2975|null|    40|  2022-01-05|
| 7782| RAVI|MANAGER|7839|1981-01-06|2450| 100|  null|  2022-01-02|
| null| null|MANAGER|null|1981-01-01|null|null|  null|  2022-01-02|
+-----+-----+-------+----+----------+----+----+------+------------+



# Apply Operations on Columns

In [ ]:
df1.select('SAL',col('SAL')/100).show()

+----+-----------+
| SAL|(SAL / 100)|
+----+-----------+
| 800|        8.0|
|1600|       16.0|
|1250|       12.5|
|2975|      29.75|
|1250|       12.5|
|2850|       28.5|
|2450|       24.5|
|null|       null|
|null|       null|
|1500|       15.0|
|1100|       11.0|
| 950|        9.5|
|3000|       30.0|
| 800|        8.0|
|1600|       16.0|
|1250|       12.5|
+----+-----------+



# Adding new column

In [ ]:
df2 = df1.withColumn('NEW_SAL',col('SAL')*100)
df2.show()

+-----+------+--------+----+----------+----+----+------+------------+-------+
|EMPNO| ENAME|     JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|NEW_SAL|
+-----+------+--------+----+----------+----+----+------+------------+-------+
| 7369| SMITH|   CLERK|7902|1980-01-17| 800| 300|    10|  2022-01-01|  80000|
| 7499| ALLEN|SALESMAN|7698|1981-01-20|1600| 300|    20|  2022-01-01| 160000|
| 7521|  WARD|SALESMAN|7698|1981-01-22|1250| 500|    30|  2022-01-01| 125000|
| 7566| JONES| MANAGER|7839|1981-01-04|2975|null|    40|  2022-01-05| 297500|
| 7654|MARTIN|SALESMAN|7698|1981-01-21|1250|1400|    50|  2022-01-03| 125000|
| 7698|   SGR|    null|7839|1981-01-05|2850|1600|    60|  2022-01-04| 285000|
| 7782|  RAVI| MANAGER|7839|1981-01-06|2450| 100|  null|  2022-01-02| 245000|
| 7788| SCOTT| ANALYST|7566|1987-01-19|null|null|    80|  2022-01-02|   null|
| null|  null| MANAGER|null|1981-01-01|null|null|  null|  2022-01-02|   null|
| 7844|TURNER|SALESMAN|7698|1981-01-09|1500|null|   100|  2022-0

#Adding new columns with different values

In [ ]:
df3 =df2.withColumn('CADER',when(col('JOB')=='CLERK','Lower').when(col('JOB')=='SALESMAN','lowest').when(col('JOB')=='MANAGER','Management').when(col('JOB')=='ANALYST','Middle'))
df3.show()

+-----+------+--------+----+----------+----+----+------+------------+-------+----------+
|EMPNO| ENAME|     JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|NEW_SAL|     CADER|
+-----+------+--------+----+----------+----+----+------+------------+-------+----------+
| 7369| SMITH|   CLERK|7902|1980-01-17| 800| 300|    10|  2022-01-01|  80000|     Lower|
| 7499| ALLEN|SALESMAN|7698|1981-01-20|1600| 300|    20|  2022-01-01| 160000|    lowest|
| 7521|  WARD|SALESMAN|7698|1981-01-22|1250| 500|    30|  2022-01-01| 125000|    lowest|
| 7566| JONES| MANAGER|7839|1981-01-04|2975|null|    40|  2022-01-05| 297500|Management|
| 7654|MARTIN|SALESMAN|7698|1981-01-21|1250|1400|    50|  2022-01-03| 125000|    lowest|
| 7698|   SGR|    null|7839|1981-01-05|2850|1600|    60|  2022-01-04| 285000|      null|
| 7782|  RAVI| MANAGER|7839|1981-01-06|2450| 100|  null|  2022-01-02| 245000|Management|
| 7788| SCOTT| ANALYST|7566|1987-01-19|null|null|    80|  2022-01-02|   null|    Middle|
| null|  null| MANAGE

# filling null values 

In [ ]:
df4 = df3.fillna(1).fillna('Missing',['ENAME','JOB'])
df4.show()
df4.count()

+-----+-------+--------+----+----------+----+----+------+------------+-------+----------+
|EMPNO|  ENAME|     JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|NEW_SAL|     CADER|
+-----+-------+--------+----+----------+----+----+------+------------+-------+----------+
| 7369|  SMITH|   CLERK|7902|1980-01-17| 800| 300|    10|  2022-01-01|  80000|     Lower|
| 7499|  ALLEN|SALESMAN|7698|1981-01-20|1600| 300|    20|  2022-01-01| 160000|    lowest|
| 7521|   WARD|SALESMAN|7698|1981-01-22|1250| 500|    30|  2022-01-01| 125000|    lowest|
| 7566|  JONES| MANAGER|7839|1981-01-04|2975|   1|    40|  2022-01-05| 297500|Management|
| 7654| MARTIN|SALESMAN|7698|1981-01-21|1250|1400|    50|  2022-01-03| 125000|    lowest|
| 7698|    SGR| Missing|7839|1981-01-05|2850|1600|    60|  2022-01-04| 285000|      null|
| 7782|   RAVI| MANAGER|7839|1981-01-06|2450| 100|     1|  2022-01-02| 245000|Management|
| 7788|  SCOTT| ANALYST|7566|1987-01-19|   1|   1|    80|  2022-01-02|      1|    Middle|
|    1|Mis

16

# dropping null values

In [ ]:
df5 = df4.dropna()
df5.show()
df5.count()

+-----+-------+--------+----+----------+----+----+------+------------+-------+----------+
|EMPNO|  ENAME|     JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|NEW_SAL|     CADER|
+-----+-------+--------+----+----------+----+----+------+------------+-------+----------+
| 7369|  SMITH|   CLERK|7902|1980-01-17| 800| 300|    10|  2022-01-01|  80000|     Lower|
| 7499|  ALLEN|SALESMAN|7698|1981-01-20|1600| 300|    20|  2022-01-01| 160000|    lowest|
| 7521|   WARD|SALESMAN|7698|1981-01-22|1250| 500|    30|  2022-01-01| 125000|    lowest|
| 7566|  JONES| MANAGER|7839|1981-01-04|2975|   1|    40|  2022-01-05| 297500|Management|
| 7654| MARTIN|SALESMAN|7698|1981-01-21|1250|1400|    50|  2022-01-03| 125000|    lowest|
| 7782|   RAVI| MANAGER|7839|1981-01-06|2450| 100|     1|  2022-01-02| 245000|Management|
| 7788|  SCOTT| ANALYST|7566|1987-01-19|   1|   1|    80|  2022-01-02|      1|    Middle|
|    1|Missing| MANAGER|   1|1981-01-01|   1|   1|     1|  2022-01-02|      1|Management|
| 7844| TU

15

# Database operations

In [ ]:
spark.sql('create database oracle_db')
spark.sql('show databases').show()

+---------+
|namespace|
+---------+
|  default|
|oracle_db|
+---------+



# Create Temp tables from dataframe

In [ ]:
#these are temporory tables ,dropped after spark session 

# temp tables doesn't belongs to any database

df4.registerTempTable('T1')

/usr/local/lib/python3.8/dist-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [ ]:
spark.sql('select * from T1').show()

+-----+-------+--------+----+----------+----+----+------+------------+-------+----------+
|EMPNO|  ENAME|     JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|NEW_SAL|     CADER|
+-----+-------+--------+----+----------+----+----+------+------------+-------+----------+
| 7369|  SMITH|   CLERK|7902|1980-01-17| 800| 300|    10|  2022-01-01|  80000|     Lower|
| 7499|  ALLEN|SALESMAN|7698|1981-01-20|1600| 300|    20|  2022-01-01| 160000|    lowest|
| 7521|   WARD|SALESMAN|7698|1981-01-22|1250| 500|    30|  2022-01-01| 125000|    lowest|
| 7566|  JONES| MANAGER|7839|1981-01-04|2975|   1|    40|  2022-01-05| 297500|Management|
| 7654| MARTIN|SALESMAN|7698|1981-01-21|1250|1400|    50|  2022-01-03| 125000|    lowest|
| 7698|    SGR| Missing|7839|1981-01-05|2850|1600|    60|  2022-01-04| 285000|      null|
| 7782|   RAVI| MANAGER|7839|1981-01-06|2450| 100|     1|  2022-01-02| 245000|Management|
| 7788|  SCOTT| ANALYST|7566|1987-01-19|   1|   1|    80|  2022-01-02|      1|    Middle|
|    1|Mis

In [ ]:
spark.sql('show tables').show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |       t1|       true|
+---------+---------+-----------+



# Creating Permenent tables from dataframe

In [ ]:
#these tables will save in databases,we access later

# Only perment tables belongs to Particular database

spark.sql('use oracle_db')

DataFrame[]

In [ ]:
spark.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |       t1|       true|
+---------+---------+-----------+



In [ ]:
#spark.sql('drop table t1')
#spark.sql('drop table Emp_data')

In [ ]:
df4.write.saveAsTable('Emp_data')


In [ ]:
spark.sql('select * from Emp_data').show()

+-----+-------+--------+----+----------+----+----+------+------------+-------+----------+
|EMPNO|  ENAME|     JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|NEW_SAL|     CADER|
+-----+-------+--------+----+----------+----+----+------+------------+-------+----------+
| 7369|  SMITH|   CLERK|7902|1980-01-17| 800| 300|    10|  2022-01-01|  80000|     Lower|
| 7499|  ALLEN|SALESMAN|7698|1981-01-20|1600| 300|    20|  2022-01-01| 160000|    lowest|
| 7521|   WARD|SALESMAN|7698|1981-01-22|1250| 500|    30|  2022-01-01| 125000|    lowest|
| 7566|  JONES| MANAGER|7839|1981-01-04|2975|   1|    40|  2022-01-05| 297500|Management|
| 7654| MARTIN|SALESMAN|7698|1981-01-21|1250|1400|    50|  2022-01-03| 125000|    lowest|
| 7698|    SGR| Missing|7839|1981-01-05|2850|1600|    60|  2022-01-04| 285000|      null|
| 7782|   RAVI| MANAGER|7839|1981-01-06|2450| 100|     1|  2022-01-02| 245000|Management|
| 7788|  SCOTT| ANALYST|7566|1987-01-19|   1|   1|    80|  2022-01-02|      1|    Middle|
|    1|Mis

# Creating Data Frame from mysql table

In [ ]:
""" 
df_mysql = spark.read.format('jdbc').\
           option('url','jdbc:mysql://localhost:3306').\
           option('driver','com.mysql.jdbc.Driver').\
           option('user','root').\
           option('password','sandeep').\
           option('query','select * from sandeep.emp_table').\
           load()
"""

" \ndf_mysql = spark.read.format('jdbc').           option('url','jdbc:mysql://localhost:3306').           option('driver','com.mysql.jdbc.Driver').           option('user','root').           option('password','sandeep').           option('query','select * from sandeep.emp_table').           load()\n"

# Json file Handling


complex Data types

1.struct - dict

2.array -  list - To flattern complex datatype(array datatype ) we can you explode() function

3.map




In [ ]:
# Creating DataFrame from Json file

data = spark.read.format('json').load('/content/emp.json')
data.show()
data.printSchema()
data.count()

+----+------+-----+------+----------+---------+----+----+------------+
|COMM|DEPTNO|EMPNO| ENAME|  HIREDATE|      JOB| MGR| SAL|UPDATED_DATE|
+----+------+-----+------+----------+---------+----+----+------------+
|  78|    80| 1234|SEKHAR|      null|   doctor|7777| 667|  2022-01-03|
|null|    20| 7369| SMITH|17-12-1980|    CLERK|7902| 800|  2022-01-01|
| 300|    30| 7499| ALLEN|20-02-1981| SALESMAN|7698|1600|  2022-01-01|
| 500|    30| 7521|  WARD|22-02-1981| SALESMAN|7698|1250|  2022-01-01|
|null|    20| 7566| JONES|04-02-1981|  MANAGER|7839|2975|  2022-01-05|
|1400|    30| 7654|MARTIN|21-09-1981| SALESMAN|7698|1250|  2022-01-03|
|null|    30| 7698|   SGR|05-01-1981|  MANAGER|7839|2850|  2022-01-04|
|null|    10| 7782|  RAVI|06-09-1981|  MANAGER|7839|2450|  2022-01-02|
|null|    20| 7788| SCOTT|19-04-1987|  ANALYST|7566|3000|  2022-01-02|
|null|    10| 7839|  KING|01-11-1981|PRESIDENT|null|5000|  2022-01-02|
|   0|    30| 7844|TURNER|09-08-1981| SALESMAN|7698|1500|  2022-01-02|
|null|

15

# Creating DataFrame from multiLine Json file

In [ ]:
# if json file has  more no.of lines (nested data) .then we should use multiLine in options

data2 = spark.read.format('json').option('multiLine',True).option('inferSchema',True).option('nullValue','null').load('/content/nested_json.json')
data2.printSchema()
data2.show(truncate = False)

root
 |-- batters: struct (nullable = true)
 |    |-- batter: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ppu: double (nullable = true)
 |-- topping: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- type: string (nullable = true)

+-------------------------------------------------------------------------------+----+----+----+-----------------------------------------------------------------------------------------------------------------------------------------+-----+
|batters                                                                        |id  |name|ppu |topping                                                                                                   

In [ ]:
data3 = data2.withColumn('topping_explode',explode('topping'))\
     .withColumn('topping_id',col('topping_explode.id'))\
     .withColumn('topping_type',col('topping_explode.type'))\
     .drop('topping_explode','topping')\
     .withColumn('batters_explode',explode('batters.batter'))\
     .withColumn('batter_id',col('batters_explode.id'))\
     .withColumn('batter_type',col("batters_explode.type"))\
     .drop('batters','batters_explode')

data3.show(truncate=False)
data3.count()

+----+----+----+-----+----------+------------------------+---------+------------+
|id  |name|ppu |type |topping_id|topping_type            |batter_id|batter_type |
+----+----+----+-----+----------+------------------------+---------+------------+
|0001|Cake|0.55|donut|5001      |None                    |1001     |Regular     |
|0001|Cake|0.55|donut|5001      |None                    |1002     |Chocolate   |
|0001|Cake|0.55|donut|5001      |None                    |1003     |Blueberry   |
|0001|Cake|0.55|donut|5001      |None                    |1004     |Devil's Food|
|0001|Cake|0.55|donut|5002      |Glazed                  |1001     |Regular     |
|0001|Cake|0.55|donut|5002      |Glazed                  |1002     |Chocolate   |
|0001|Cake|0.55|donut|5002      |Glazed                  |1003     |Blueberry   |
|0001|Cake|0.55|donut|5002      |Glazed                  |1004     |Devil's Food|
|0001|Cake|0.55|donut|5005      |Sugar                   |1001     |Regular     |
|0001|Cake|0.55|

28

# Creating Data Frame from REST API 

In [ ]:
#REST API -Accessing the data over internet through Urls

import requests
import json


api = requests.request('GET','https://api.github.com/users/hadley/orgs')

data = api.json()

file = open('/content/sample_data/apidata.json','a')

for record in data:
  file.write("%s\n" %record)
 
api_df = spark.read.format('json').load('/content/sample_data/apidata.json')


In [ ]:
print(type(api.json()))
print(len(api.json()))

<class 'list'>
10


In [ ]:
api_df.show(10)
api_df.printSchema()
api_df.count()

++
||
++
++

root



0

# Functions in pyspark

In [ ]:
from pyspark.sql.functions import *

fun = spark.sql('show functions')
print(fun.count())
print(fun.show(388))

388
+--------------------+
|            function|
+--------------------+
|                   !|
|                  !=|
|                   %|
|                   &|
|                   *|
|                   +|
|                   -|
|                   /|
|                   <|
|                  <=|
|                 <=>|
|                  <>|
|                   =|
|                  ==|
|                   >|
|                  >=|
|                   ^|
|                 abs|
|                acos|
|               acosh|
|          add_months|
|         aes_decrypt|
|         aes_encrypt|
|           aggregate|
|                 and|
|                 any|
|approx_count_dist...|
|   approx_percentile|
|               array|
|           array_agg|
|      array_contains|
|      array_distinct|
|        array_except|
|     array_intersect|
|          array_join|
|           array_max|
|           array_min|
|      array_position|
|        array_remove|
|        array_repeat|
|      

In [ ]:
print(type(fun))

# describe function details
spark.sql('describe function aggregate').show(truncate=False)

<class 'pyspark.sql.dataframe.DataFrame'>
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|function_desc                                                                                                                                                                                                                                                                |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Function: aggregate                                                                                                                          

# HIVE vs RDBMS

Hive 

- Hive uses HQL/SQL langague 
- Hive uses HDFS  as storage 
- Hive used for Analysis
- Metadata stored in Meta Store --- a database in RDBMS

RDBMS

- Oracle db uses Sql langague
- oracle uses NTFS/EXT4 etc
- used for real time transcations
- metadata stored in RDBMS 

# Creating dataframe from Hive table



In [ ]:

#DSL - Domain Specific language 


df_hive = spark.read.table('oracle_db.emp_data')

print(df_hive.count())

df_hive.show()


16
+-----+-------+--------+----+----------+----+----+------+------------+-------+----------+
|EMPNO|  ENAME|     JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|NEW_SAL|     CADER|
+-----+-------+--------+----+----------+----+----+------+------------+-------+----------+
| 7369|  SMITH|   CLERK|7902|1980-01-17| 800| 300|    10|  2022-01-01|  80000|     Lower|
| 7499|  ALLEN|SALESMAN|7698|1981-01-20|1600| 300|    20|  2022-01-01| 160000|    lowest|
| 7521|   WARD|SALESMAN|7698|1981-01-22|1250| 500|    30|  2022-01-01| 125000|    lowest|
| 7566|  JONES| MANAGER|7839|1981-01-04|2975|   1|    40|  2022-01-05| 297500|Management|
| 7654| MARTIN|SALESMAN|7698|1981-01-21|1250|1400|    50|  2022-01-03| 125000|    lowest|
| 7698|    SGR| Missing|7839|1981-01-05|2850|1600|    60|  2022-01-04| 285000|      null|
| 7782|   RAVI| MANAGER|7839|1981-01-06|2450| 100|     1|  2022-01-02| 245000|Management|
| 7788|  SCOTT| ANALYST|7566|1987-01-19|   1|   1|    80|  2022-01-02|      1|    Middle|
|    1|

In [ ]:

# pure sql language

df_hive1 = spark.sql("select * from oracle_db.emp_data")

print(df_hive1.count())
df_hive1.show()


16
+-----+-------+--------+----+----------+----+----+------+------------+-------+----------+
|EMPNO|  ENAME|     JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|NEW_SAL|     CADER|
+-----+-------+--------+----+----------+----+----+------+------------+-------+----------+
| 7369|  SMITH|   CLERK|7902|1980-01-17| 800| 300|    10|  2022-01-01|  80000|     Lower|
| 7499|  ALLEN|SALESMAN|7698|1981-01-20|1600| 300|    20|  2022-01-01| 160000|    lowest|
| 7521|   WARD|SALESMAN|7698|1981-01-22|1250| 500|    30|  2022-01-01| 125000|    lowest|
| 7566|  JONES| MANAGER|7839|1981-01-04|2975|   1|    40|  2022-01-05| 297500|Management|
| 7654| MARTIN|SALESMAN|7698|1981-01-21|1250|1400|    50|  2022-01-03| 125000|    lowest|
| 7698|    SGR| Missing|7839|1981-01-05|2850|1600|    60|  2022-01-04| 285000|      null|
| 7782|   RAVI| MANAGER|7839|1981-01-06|2450| 100|     1|  2022-01-02| 245000|Management|
| 7788|  SCOTT| ANALYST|7566|1987-01-19|   1|   1|    80|  2022-01-02|      1|    Middle|
|    1|

# Save Dataframe to Hive

In [ ]:
df_final = df_hive.withColumn('Date',current_timestamp())

df_final.write.partitionBy('Date').saveAsTable('oracle_db.new_emp_snap_view')


In [ ]:
spark.sql('select * from new_emp_snap_view').show()

+-----+-------+--------+----+----------+----+----+------+------------+-------+----------+--------------------+
|EMPNO|  ENAME|     JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|NEW_SAL|     CADER|                Date|
+-----+-------+--------+----+----------+----+----+------+------------+-------+----------+--------------------+
| 7369|  SMITH|   CLERK|7902|1980-01-17| 800| 300|    10|  2022-01-01|  80000|     Lower|2023-03-08 14:56:...|
| 7499|  ALLEN|SALESMAN|7698|1981-01-20|1600| 300|    20|  2022-01-01| 160000|    lowest|2023-03-08 14:56:...|
| 7521|   WARD|SALESMAN|7698|1981-01-22|1250| 500|    30|  2022-01-01| 125000|    lowest|2023-03-08 14:56:...|
| 7566|  JONES| MANAGER|7839|1981-01-04|2975|   1|    40|  2022-01-05| 297500|Management|2023-03-08 14:56:...|
| 7654| MARTIN|SALESMAN|7698|1981-01-21|1250|1400|    50|  2022-01-03| 125000|    lowest|2023-03-08 14:56:...|
| 7698|    SGR| Missing|7839|1981-01-05|2850|1600|    60|  2022-01-04| 285000|      null|2023-03-08 14:56:...|
|

#  SPARK Input & OutPut formats



Bydefault spark will read files in snappy.parquet

Bydefault spark will save out files in snappy.parquet

User need to define input and output formats By using format Function

# word count program step by step

In [ ]:
rd = spark.sparkContext.textFile("/content/word.txt")

print(type(rd))
print(rd.collect())       # no.of lines
print(rd.count())

<class 'pyspark.rdd.RDD'>
['The forest raven also commonly known as the Tasmanian raven', 'is a passerine bird in the family Corvidae native to Tasmania and parts of southern Victoria', 'such as Wilsons Promontory and Portland Populations are also found in parts of New South Wales', 'including Dorrigo and Armidale it has allblack plumage', 'beak and legs As with the other two species of raven in Australia, its black feathers have grey bases', 'Adults have white irises; younger birds have dark brown and then hazel irises with an inner blue rim', 'New South Wales populations are recognised as a separate subspecies C tasmanicus boreus', 'but appear to be nested within the Tasmanian subspecies genetically']
8


In [ ]:
rd1 = rd.map(lambda x : x.encode('utf-8'))

print(rd1.collect())

[b'The forest raven also commonly known as the Tasmanian raven', b'is a passerine bird in the family Corvidae native to Tasmania and parts of southern Victoria', b'such as Wilsons Promontory and Portland Populations are also found in parts of New South Wales', b'including Dorrigo and Armidale it has allblack plumage', b'beak and legs As with the other two species of raven in Australia, its black feathers have grey bases', b'Adults have white irises; younger birds have dark brown and then hazel irises with an inner blue rim', b'New South Wales populations are recognised as a separate subspecies C tasmanicus boreus', b'but appear to be nested within the Tasmanian subspecies genetically']


In [ ]:
# map - gives Number of lists of strings

rd2 = rd.map(lambda x:x.split(' '))
print(type(rd2))
print(rd2.collect())

<class 'pyspark.rdd.PipelinedRDD'>
[['The', 'forest', 'raven', 'also', 'commonly', 'known', 'as', 'the', 'Tasmanian', 'raven'], ['is', 'a', 'passerine', 'bird', 'in', 'the', 'family', 'Corvidae', 'native', 'to', 'Tasmania', 'and', 'parts', 'of', 'southern', 'Victoria'], ['such', 'as', 'Wilsons', 'Promontory', 'and', 'Portland', 'Populations', 'are', 'also', 'found', 'in', 'parts', 'of', 'New', 'South', 'Wales'], ['including', 'Dorrigo', 'and', 'Armidale', 'it', 'has', 'allblack', 'plumage'], ['beak', 'and', 'legs', 'As', 'with', 'the', 'other', 'two', 'species', 'of', 'raven', 'in', 'Australia,', 'its', 'black', 'feathers', 'have', 'grey', 'bases'], ['Adults', 'have', 'white', 'irises;', 'younger', 'birds', 'have', 'dark', 'brown', 'and', 'then', 'hazel', 'irises', 'with', 'an', 'inner', 'blue', 'rim'], ['New', 'South', 'Wales', 'populations', 'are', 'recognised', 'as', 'a', 'separate', 'subspecies', 'C', 'tasmanicus', 'boreus'], ['but', 'appear', 'to', 'be', 'nested', 'within', 'the',

In [ ]:
#flatmap - gives single list of strings

rd3 = rd.flatMap(lambda x : x.split(' '))
print(rd3.collect())

['The', 'forest', 'raven', 'also', 'commonly', 'known', 'as', 'the', 'Tasmanian', 'raven', 'is', 'a', 'passerine', 'bird', 'in', 'the', 'family', 'Corvidae', 'native', 'to', 'Tasmania', 'and', 'parts', 'of', 'southern', 'Victoria', 'such', 'as', 'Wilsons', 'Promontory', 'and', 'Portland', 'Populations', 'are', 'also', 'found', 'in', 'parts', 'of', 'New', 'South', 'Wales', 'including', 'Dorrigo', 'and', 'Armidale', 'it', 'has', 'allblack', 'plumage', 'beak', 'and', 'legs', 'As', 'with', 'the', 'other', 'two', 'species', 'of', 'raven', 'in', 'Australia,', 'its', 'black', 'feathers', 'have', 'grey', 'bases', 'Adults', 'have', 'white', 'irises;', 'younger', 'birds', 'have', 'dark', 'brown', 'and', 'then', 'hazel', 'irises', 'with', 'an', 'inner', 'blue', 'rim', 'New', 'South', 'Wales', 'populations', 'are', 'recognised', 'as', 'a', 'separate', 'subspecies', 'C', 'tasmanicus', 'boreus', 'but', 'appear', 'to', 'be', 'nested', 'within', 'the', 'Tasmanian', 'subspecies', 'genetically']


In [ ]:
# assiging value to strings
rd4 =  rd3.map(lambda x : (x,1))
print(rd4.collect())

[('The', 1), ('forest', 1), ('raven', 1), ('also', 1), ('commonly', 1), ('known', 1), ('as', 1), ('the', 1), ('Tasmanian', 1), ('raven', 1), ('is', 1), ('a', 1), ('passerine', 1), ('bird', 1), ('in', 1), ('the', 1), ('family', 1), ('Corvidae', 1), ('native', 1), ('to', 1), ('Tasmania', 1), ('and', 1), ('parts', 1), ('of', 1), ('southern', 1), ('Victoria', 1), ('such', 1), ('as', 1), ('Wilsons', 1), ('Promontory', 1), ('and', 1), ('Portland', 1), ('Populations', 1), ('are', 1), ('also', 1), ('found', 1), ('in', 1), ('parts', 1), ('of', 1), ('New', 1), ('South', 1), ('Wales', 1), ('including', 1), ('Dorrigo', 1), ('and', 1), ('Armidale', 1), ('it', 1), ('has', 1), ('allblack', 1), ('plumage', 1), ('beak', 1), ('and', 1), ('legs', 1), ('As', 1), ('with', 1), ('the', 1), ('other', 1), ('two', 1), ('species', 1), ('of', 1), ('raven', 1), ('in', 1), ('Australia,', 1), ('its', 1), ('black', 1), ('feathers', 1), ('have', 1), ('grey', 1), ('bases', 1), ('Adults', 1), ('have', 1), ('white', 1), 

In [ ]:
# reduceByKey -  groupping and aggregating same keys from rd4

rd5 = rd4.reduceByKey(lambda x ,y : x+y)

print(rd5.collect())

[('The', 1), ('forest', 1), ('raven', 3), ('also', 2), ('commonly', 1), ('known', 1), ('as', 3), ('the', 4), ('Tasmanian', 2), ('is', 1), ('a', 2), ('passerine', 1), ('bird', 1), ('in', 3), ('family', 1), ('Corvidae', 1), ('native', 1), ('to', 2), ('Tasmania', 1), ('and', 5), ('parts', 2), ('of', 3), ('southern', 1), ('Victoria', 1), ('such', 1), ('Wilsons', 1), ('Promontory', 1), ('Portland', 1), ('Populations', 1), ('are', 2), ('found', 1), ('New', 2), ('South', 2), ('Wales', 2), ('including', 1), ('Dorrigo', 1), ('Armidale', 1), ('it', 1), ('has', 1), ('allblack', 1), ('plumage', 1), ('beak', 1), ('legs', 1), ('As', 1), ('with', 2), ('other', 1), ('two', 1), ('species', 1), ('Australia,', 1), ('its', 1), ('black', 1), ('feathers', 1), ('have', 3), ('grey', 1), ('bases', 1), ('Adults', 1), ('white', 1), ('irises;', 1), ('younger', 1), ('birds', 1), ('dark', 1), ('brown', 1), ('then', 1), ('hazel', 1), ('irises', 1), ('an', 1), ('inner', 1), ('blue', 1), ('rim', 1), ('populations', 1)

# word count in single line - Find No.of Occurences of single word

In [ ]:
R = spark.sparkContext.textFile('/content/word.txt').flatMap(lambda x : x.split(' ')).map(lambda x: (x,1)).reduceByKey(lambda x ,y : x+y)
R.collect()

[('The', 1),
 ('forest', 1),
 ('raven', 3),
 ('also', 2),
 ('commonly', 1),
 ('known', 1),
 ('as', 3),
 ('the', 4),
 ('Tasmanian', 2),
 ('is', 1),
 ('a', 2),
 ('passerine', 1),
 ('bird', 1),
 ('in', 3),
 ('family', 1),
 ('Corvidae', 1),
 ('native', 1),
 ('to', 2),
 ('Tasmania', 1),
 ('and', 5),
 ('parts', 2),
 ('of', 3),
 ('southern', 1),
 ('Victoria', 1),
 ('such', 1),
 ('Wilsons', 1),
 ('Promontory', 1),
 ('Portland', 1),
 ('Populations', 1),
 ('are', 2),
 ('found', 1),
 ('New', 2),
 ('South', 2),
 ('Wales', 2),
 ('including', 1),
 ('Dorrigo', 1),
 ('Armidale', 1),
 ('it', 1),
 ('has', 1),
 ('allblack', 1),
 ('plumage', 1),
 ('beak', 1),
 ('legs', 1),
 ('As', 1),
 ('with', 2),
 ('other', 1),
 ('two', 1),
 ('species', 1),
 ('Australia,', 1),
 ('its', 1),
 ('black', 1),
 ('feathers', 1),
 ('have', 3),
 ('grey', 1),
 ('bases', 1),
 ('Adults', 1),
 ('white', 1),
 ('irises;', 1),
 ('younger', 1),
 ('birds', 1),
 ('dark', 1),
 ('brown', 1),
 ('then', 1),
 ('hazel', 1),
 ('irises', 1),
 ('a

# Spark default Ram and Cores

In [ ]:
# default ram is  - 1 GB
# default cores are - local/local[1] - 1 core will be added/allocated
#                   - local[n] - n cores will be added/allocated
#                   - local[*] - all available cores in cluster 

# from pyspark.sql import sparkSession
# spark = sparkSession.builder.master(local[*]).appName('demo').getOrCreate()

# fill missing data in textFile and convert into Dataframe

In [ ]:
df_miss = spark.read.format('csv').load('/content/fill missing.txt',sep = ' ').fillna('no data')

df_miss.show()


+-----+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|  _c0|    _c1|    _c2|    _c3|    _c4|    _c5|    _c6|    _c7|    _c8|    _c9|   _c10|   _c11|
+-----+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|Spark|  Spark|  Spark|  Spark|  Spark|  Spark|  Spark|  Spark|  Spark|  Spark|  Spark|no data|
|Spark|  Spark|  Spark|  Spark|  Spark|  Spark|  Spark|  Spark|  Spark|  Spark|no data|no data|
|Spark|  Spark|  Spark|  Spark|  Spark|  Spark|  Spark|  Spark|  Spark|no data|no data|no data|
|Spark|  Spark|  Spark|  Spark|  Spark|  Spark|  Spark|  Spark|no data|no data|no data|no data|
|Spark|  Spark|  Spark|  Spark|  Spark|  Spark|  Spark|no data|no data|no data|no data|no data|
|Spark|  Spark|  Spark|  Spark|  Spark|  Spark|no data|no data|no data|no data|no data|no data|
|Spark|  Spark|  Spark|  Spark|  Spark|no data|no data|no data|no data|no data|no data|no data|
|Spark|  Spark|  Spark|  Spark|no data|n

#udf register

In [ ]:
from pyspark.sql.types import *

# UDF 

def f1(x):
  return ((x*x)-x)

spark.udf.register('fun',f1,IntegerType())
fun = udf(f1,IntegerType())

print(f1(5))

20


In [ ]:
df5.select('ENAME',sqrt('SAL'),fun('SAL')).show()

+-------+------------------+-------+
|  ENAME|         SQRT(SAL)|f1(SAL)|
+-------+------------------+-------+
|  SMITH|28.284271247461902| 639200|
|  ALLEN|              40.0|2558400|
|   WARD| 35.35533905932738|1561250|
|  JONES|54.543560573178574|8847650|
| MARTIN| 35.35533905932738|1561250|
|   RAVI| 49.49747468305833|6000050|
|  SCOTT|               1.0|      0|
|Missing|               1.0|      0|
| TURNER| 38.72983346207417|2248500|
|  ADAMS|   33.166247903554|1208900|
|  JAMES|30.822070014844883| 901550|
|   FORD|54.772255750516614|8997000|
|  SMITH|28.284271247461902| 639200|
|  ALLEN|              40.0|2558400|
|   WARD| 35.35533905932738|1561250|
+-------+------------------+-------+



# how to handle yy date format in pyspark for before 2000 data

In [ ]:
yy_df = spark.read.format('csv').option('header',True).option('sep','|').option('inferSchema',True).option('nullValue','null').load('/content/emp_pipe_yy.txt')

yy_df.show()
yy_df.printSchema()

+-----+------+---------+----+----+----+------+------------+
|EMPNO| ENAME|      JOB| MGR| SAL|COMM|DEPTNO|UPDATED_DATE|
+-----+------+---------+----+----+----+------+------------+
| 7369| SMITH|    CLERK|7902| 800|null|    20|    01-01-98|
| 7499| ALLEN| SALESMAN|7698|1600| 300|    30|    02-01-97|
| 7521|  WARD| SALESMAN|7698|1250| 500|    30|    03-01-96|
| 7566| JONES|  MANAGER|7839|2975|null|    20|    04-01-95|
| 7654|MARTIN| SALESMAN|7698|1250|1400|    30|    05-01-98|
| 7698|   SGR|  MANAGER|7839|2850|null|    30|    06-01-97|
| 7782|  RAVI|  MANAGER|7839|2450|null|    10|    07-01-96|
| 7788| SCOTT|  ANALYST|7566|3000|null|    20|    08-01-95|
| 7839|  KING|PRESIDENT|null|5000|null|    10|        null|
| 7844|TURNER| SALESMAN|7698|1500|   0|    30|    01-02-80|
| 7876| ADAMS|    CLERK|7788|1100|null|    20|    02-02-81|
| 7900| JAMES|    CLERK|7698| 950|null|    30|    03-02-82|
| 7902|  FORD|  ANALYST|7566|3000|null|    20|    04-02-83|
| 7934|MILLER|    CLERK|7782|1300|null| 

In [ ]:

#Spark defaulty chooses year after 2000 

# we need to set spark sql legcy timeparserpolicy to legacy -- Useful for less columns data only ,if more data means need ask source system set proper date format

spark.conf.set('spark.sql.legacy.timeParserPolicy','LEGACY')

yy_df.withColumn('DATE',to_date('UPDATED_DATE','dd-mm-yy')).show()

+-----+------+---------+----+----+----+------+------------+----------+
|EMPNO| ENAME|      JOB| MGR| SAL|COMM|DEPTNO|UPDATED_DATE|      DATE|
+-----+------+---------+----+----+----+------+------------+----------+
| 7369| SMITH|    CLERK|7902| 800|null|    20|    01-01-98|1998-01-01|
| 7499| ALLEN| SALESMAN|7698|1600| 300|    30|    02-01-97|1997-01-02|
| 7521|  WARD| SALESMAN|7698|1250| 500|    30|    03-01-96|1996-01-03|
| 7566| JONES|  MANAGER|7839|2975|null|    20|    04-01-95|1995-01-04|
| 7654|MARTIN| SALESMAN|7698|1250|1400|    30|    05-01-98|1998-01-05|
| 7698|   SGR|  MANAGER|7839|2850|null|    30|    06-01-97|1997-01-06|
| 7782|  RAVI|  MANAGER|7839|2450|null|    10|    07-01-96|1996-01-07|
| 7788| SCOTT|  ANALYST|7566|3000|null|    20|    08-01-95|1995-01-08|
| 7839|  KING|PRESIDENT|null|5000|null|    10|        null|      null|
| 7844|TURNER| SALESMAN|7698|1500|   0|    30|    01-02-80|1980-01-01|
| 7876| ADAMS|    CLERK|7788|1100|null|    20|    02-02-81|1981-01-02|
| 7900

# how to handle duplicate column error

In [ ]:
dept = spark.read.format('csv').option('header',True).option('inferSchema',True).option('nullValue','null').load('/content/dept.csv')
emp = spark.read.format('csv').option('header',True).option('inferSchema',True).option('nullValue','null').load('/content/emp.csv')

dept.show()
emp.show()

+----------+--------+-----+
|     DNAME|     LOC|depno|
+----------+--------+-----+
|ACCOUNTING|NEW YORK|   10|
|  RESEARCH|  DALLAS|   20|
|     SALES| CHICAGO|   30|
|OPERATIONS|  BOSTON|   40|
|ACCOUNTING|NEW YORK|   50|
|  RESEARCH|  DALLAS|   60|
|     SALES| CHICAGO|   70|
|OPERATIONS|  BOSTON|   80|
|ACCOUNTING|NEW YORK|   90|
|  RESEARCH|  DALLAS|  100|
|     SALES| CHICAGO|   10|
|OPERATIONS|  BOSTON|   20|
|ACCOUNTING|NEW YORK|   30|
|  RESEARCH|  DALLAS|   40|
|     SALES| CHICAGO|   50|
|OPERATIONS|  BOSTON|   60|
+----------+--------+-----+

+-----+------+--------+----+----------+----+----+------+------------+
|EMPNO| ENAME|     JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|
+-----+------+--------+----+----------+----+----+------+------------+
| 7369| SMITH|   CLERK|7902|17-12-1980| 800| 300|    10|  01-01-2022|
| 7499| ALLEN|SALESMAN|7698|20-02-1981|1600| 300|    20|  01-01-2022|
| 7521|  WARD|SALESMAN|7698|22-02-1981|1250| 500|    30|  01-01-2022|
| 7566| JONES| MANA

In [ ]:
# innerjoin 

emp_dept = emp.join(dept,emp['DEPTNO'] == dept['depno'],'inner').drop('depno')

emp_dept.show()




+-----+------+--------+----+----------+----+----+------+------------+----------+--------+
|EMPNO| ENAME|     JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|     DNAME|     LOC|
+-----+------+--------+----+----------+----+----+------+------------+----------+--------+
| 7369| SMITH|   CLERK|7902|17-12-1980| 800| 300|    10|  01-01-2022|     SALES| CHICAGO|
| 7369| SMITH|   CLERK|7902|17-12-1980| 800| 300|    10|  01-01-2022|ACCOUNTING|NEW YORK|
| 7499| ALLEN|SALESMAN|7698|20-02-1981|1600| 300|    20|  01-01-2022|OPERATIONS|  BOSTON|
| 7499| ALLEN|SALESMAN|7698|20-02-1981|1600| 300|    20|  01-01-2022|  RESEARCH|  DALLAS|
| 7521|  WARD|SALESMAN|7698|22-02-1981|1250| 500|    30|  01-01-2022|ACCOUNTING|NEW YORK|
| 7521|  WARD|SALESMAN|7698|22-02-1981|1250| 500|    30|  01-01-2022|     SALES| CHICAGO|
| 7566| JONES| MANAGER|7839|04-02-1981|2975|null|    40|  05-01-2022|  RESEARCH|  DALLAS|
| 7566| JONES| MANAGER|7839|04-02-1981|2975|null|    40|  05-01-2022|OPERATIONS|  BOSTON|
| 7654|MAR

In [ ]:

# write into a delta table

emp_dept.write.saveAsTable('emp_dept_table')

In [ ]:
spark.sql('select count(*) from emp_dept_table').show()

+--------+
|count(1)|
+--------+
|      26|
+--------+



# how to handle bad data



In [ ]:
bad = spark.read.format('csv').option('header',True).option('inferSchema',True).option('nullValue','null').load('/content/channels.csv')

bad.show()

bad.schema

+--------------------+------------+-------------+----------------+-------------+----------------+
|          CHANNEL_ID|CHANNEL_DESC|CHANNEL_CLASS|CHANNEL_CLASS_ID|CHANNEL_TOTAL|CHANNEL_TOTAL_ID|
+--------------------+------------+-------------+----------------+-------------+----------------+
|                   3|Direct Sales|       Direct|              12|Channel total|               1|
|                   9|  Tele Sales|       Direct|              12|Channel total|               1|
|                   5|     Catalog|     Indirect|              13|Channel total|               1|
|                   4|    Internet|     Indirect|              13|Channel total|               1|
|              sample|    Partners|       Others|              14|Channel total|               1|
|10 Partners Other...|        null|         null|            null|         null|            null|
|11 Partners Other...|        null|         null|            null|         null|            null|
+-------------------

StructType([StructField('CHANNEL_ID', StringType(), True), StructField('CHANNEL_DESC', StringType(), True), StructField('CHANNEL_CLASS', StringType(), True), StructField('CHANNEL_CLASS_ID', IntegerType(), True), StructField('CHANNEL_TOTAL', StringType(), True), StructField('CHANNEL_TOTAL_ID', IntegerType(), True)])

Spark read Mode:

1.PERMISSIVE - allows bad data - it's spark default mode

2.FAILFAST - won't allows bad data -it raise expection - it won't process further

3.DROPMALFORMED - drops bad records based on schema -it won't save bad records

4.badrecordsPath - save good data in table and saves bad it another path

In [ ]:
from pyspark.sql.types import *

# need to add _corrupt_record column string type in custom made schema
schema  = StructType([StructField('CHANNEL_ID', IntegerType(), True), 
                      StructField('CHANNEL_DESC', StringType(), True), 
                      StructField('CHANNEL_CLASS', StringType(), True), 
                      StructField('CHANNEL_CLASS_ID', IntegerType(), True), 
                      StructField('CHANNEL_TOTAL', StringType(), True), 
                      StructField('CHANNEL_TOTAL_ID', IntegerType(), True),
                      StructField("BadData", StringType(), True)])

In [ ]:
#save bad Records Using mode - PERMISSIVE and _corrupt_record ,columnNameofCorrputRecord

bad1 = spark.read.format('csv').schema(schema).option('Mode','PERMISSIVE').option('ColumnNameOfCorruptRecord','BadData').option('header',True).option('nullValue','null').load('/content/channels.csv')
bad1.show()

# filter good records
goodData = bad1.filter('BadData is Null').drop('BAdData')
goodData.show()

# filter corrupt records
bad3 = bad1.filter('BadData is Not Null')
bad3.show()

+----------+------------+-------------+----------------+-------------+----------------+--------------------+
|CHANNEL_ID|CHANNEL_DESC|CHANNEL_CLASS|CHANNEL_CLASS_ID|CHANNEL_TOTAL|CHANNEL_TOTAL_ID|             BadData|
+----------+------------+-------------+----------------+-------------+----------------+--------------------+
|         3|Direct Sales|       Direct|              12|Channel total|               1|                null|
|         9|  Tele Sales|       Direct|              12|Channel total|               1|                null|
|         5|     Catalog|     Indirect|              13|Channel total|               1|                null|
|         4|    Internet|     Indirect|              13|Channel total|               1|                null|
|      null|    Partners|       Others|              14|Channel total|               1|sample,Partners,O...|
|      null|        null|         null|            null|         null|            null|10 Partners Other...|
|      null|       

In [ ]:
#mode - FAILFAST

bad = spark.read.format('csv').schema(schema).option('mode','FAILFAST').option('header',True).option('nullValue','null').load('/content/channels.csv')
bad.show()


Py4JJavaError: ignored

In [ ]:
#DROPMALFORMED

bad = spark.read.format('csv').schema(schema).option('mode','DROPMALFORMED').option('header',True).option('nullValue','null').load('/content/channels.csv')
bad.show()

# Difference Between sort and order by



1. Spark_sql : orderBy will do sorting an entire data ,sortby will do Partition wise sorting in sparksql .


2. pyspark : orderBy and sort are same pyspark.sortwithinpartitions same as sortby ( it will do Partition wise sorting)



In [ ]:
ta_df = spark.read.load('/content/spark-warehouse/oracle_db.db/emp_dept_table').repartition(4,'DEPTNO').withColumn('partition',spark_partition_id())

ta_df.show()

In [ ]:
#In Pyspark -orderBy and sort are same pyspark

#orderBy

ta_df.orderBy('SAL').show()

In [ ]:
#sort
ta_df.sort('SAL').show()

In [ ]:
#sortWithinPartitions -sortwithinpartitions same as sortby ( it will do Partition wise sorting)

ta_df.sortWithinPartitions('SAL').show()

In [ ]:
ta_df.createOrReplaceTempView('ta_df')

In [ ]:
#order by - sort entire data 

spark.sql('select * from ta_df order by SAL').show()

In [ ]:
#sort by - will do partition wise sorting
spark.sql('select * from ta_df sort by SAL').show()

# coalesce and repartition in rdd 

coalesce : is a  Narrow transformation : adjust data in existing partition,No shuffling ,By defult it will used for decrease the partitions.
for increasing partitions we need provide another argument True ,then it will shuffle the data.

repartition : is a wide transformation : create new partitions,Data shuffle will happen,used for increase/decrease the partitions


In [ ]:

from pyspark import SparkContext

sc = SparkContext.getOrCreate()

In [ ]:
rdd = sc.parallelize(range(10),5)

rdd1 = rdd.coalesce(2) # used to decrease no.of partitions ,No shuffle will happen

rdd2 = rdd.coalesce(4,True)  # use True to increase no.of partitions ,shuffle will happen

rdd3 = rdd.repartition(2)    # use True to decrease no.of partitions ,shuffle will happen

rdd4 = rdd.repartition(6)   # use True to increase no.of partitions ,shuffle will happen


print('original rdd', rdd.glom().collect())
print('coalesce 2 ',rdd1.glom().collect())
print('coalesce 4',rdd2.glom().collect())
print('repartition 2',rdd3.glom().collect())
print('repartition 6',rdd4.glom().collect())

# coalesce and repartition in dataframe

coalesce : is a  Narrow transformation : adjust data in existing partition,No shuffling ,By defult it will used for decrease the partitions.


repartition : is a wide transformation : create new partitions,Data shuffle will happen,used for increase/decrease the partitions,
we can repartition based on column specific to increse the performence 

In [ ]:
cr_df = spark.read.load('/content/spark-warehouse/oracle_db.db/emp_dept_table')
print(cr_df.rdd.getNumPartitions())
cr_df.show()

In [ ]:
cr_df1 = cr_df.repartition(4).withColumn('partition_id',spark_partition_id())
cr_df1.show()

In [ ]:

#repartition based on joining columns/filtering column to imporve performance
cr_df2 = cr_df.repartition(4,'DEPTNO').withColumn('partition_id',spark_partition_id())
cr_df2.show()

In [ ]:
cr_df3 = cr_df.coalesce(3).withColumn('partition_id',spark_partition_id())
cr_df3.show()